# Ejemplo de uso

In [1]:
import mib_v2_3_2 as mb

## Modelo de un problema

$P(ABC) = P(A)P(B|A)P(C|AB)$.

### Probabilidades

<table width="90%">
    <tr>
        <td valign="top">
            <table width="50%">
              <tr>
                <th> $A$</th>
                <th><p style="text-align:center;"> $P(A)$</p></th>
              </tr>
              <tr>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
              </tr>
            </table>
        </td>
        <td valign="top">
            <table width="60%">
              <tr>
                <td style="height:1px;"> &nbsp;</td>
                <td colspan="2"><p style="text-align:center;">$P$($B$|$A$)</p></td>
              </tr>
              <tr>
                <td>$A$</td>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
                <td><p style="text-align:center;">$0.8$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
              </tr>
            </table>        
        </td>
        <td valign="top">
            <table width="90%">
              <tr>
                <td style="height:1px;"> &nbsp;</td>
                <td colspan="2"><p style="text-align:center;">$P$($C$|$AB$)</p></td>
              </tr>
              <tr>
                <td>$AB$</td>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$2$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$00$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
                <td><p style="text-align:center;">$0.8$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$01$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
                <td><p style="text-align:center;">$0.5$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$10$</p></td>
                <td><p style="text-align:center;">$0.4$</p></td>
                <td><p style="text-align:center;">$0.5$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$11$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
              </tr>
            </table>        
        </td>
    </tr>
</table>

## Creación de datos

### Declaración de diccionarios

In [2]:
# Tablas
# {(A,): P(A)}
dA = {
    (0,): 0.3,
    (1,): 0.7
}

# {(variables independientes): {variables: p}}

# {(A,):{(B,): P(B|A)}}
dB_A = {
    (0,):{(0,): 0.2, (1,): 0.8},
    (1,):{(0,): 0.3, (1,): 0.7}
}

# {(A,B):{(C,): P(C|A,B)}}
dC_AB = {
    (0,0): {(0,): 0.1, (1,): 0.8, (2,): 0.1},
    (0,1): {(0,): 0.3, (1,): 0.5, (2,): 0.2},
    (1,0): {(0,): 0.4, (1,): 0.5, (2,): 0.1},
    (1,1): {(0,): 0.1, (1,): 0.7, (2,): 0.2}
    }


### Declaración de variables

In [3]:
A = mb.Var(name='A', values=set([0,1]))
B = mb.Var(name='B', values=set([0,1]))
C = mb.Var(name='C', values=set([0,1,2]))

### Declaración de distribuciones 

In [4]:
# P(A)
PA = mb.Distrib(table=dA, vars=(A,))
# P(B|A)
PB_A = mb.Distrib(table=dB_A, vars=(B,), parents=(A,))
# P(C|AB)
PC_AB = mb.Distrib(table=dC_AB, vars=(C,), parents=(A, B))

## Creación de la pregunta

In [5]:
# Probabilidad conjunta (Especificación)
PABC = mb.Specification(set([A,B,C]), (PA, PB_A, PC_AB))
# Generador de consultas (Pregunta)
QABC = mb.Question(PABC)

## Creación de consultas

### Distribuciones

#### De una variable

In [6]:
PB = QABC.DistributionQuery(set([B]))
PB.table

{(0,): 0.27, (1,): 0.7299999999999999}

In [7]:
PB = QABC.DistributionQuery(set([B]), aproximation=True)
PB.table

{(0,): 0.2748, (1,): 0.7322}

#### Conjuntas

In [8]:
PAB = QABC.DistributionQuery(set([A,B]))
PAB.table

{(0, 0): 0.06, (0, 1): 0.24, (1, 0): 0.21, (1, 1): 0.48999999999999994}

In [9]:
PAB = QABC.DistributionQuery(set([A,B]), aproximation=True)
PAB.table

{(0, 0): 0.063, (0, 1): 0.241, (1, 0): 0.2094, (1, 1): 0.4881}

#### Condicional

In [10]:
PA_B = QABC.DistributionQuery(set([A]), set([B]))
parents_names = tuple([v.name for v in PA_B.parents])
vars_names = tuple([v.name for v in PA_B.vars])
print(parents_names, vars_names)
PA_B.table

('B',) ('A',)


{(0,): {(0,): 0.2222222222222222, (1,): 0.7777777777777777},
 (1,): {(0,): 0.32876712328767127, (1,): 0.6712328767123288}}

In [11]:
PA_B = QABC.DistributionQuery(set([A]), set([B]), aproximation=True)
parents_names = tuple([v.name for v in PA_B.parents])
vars_names = tuple([v.name for v in PA_B.vars])
print(parents_names, vars_names)
PA_B.table

('B',) ('A',)


{(0,): {(0,): 0, (1,): 0}, (1,): {(0,): 0, (1,): 0}}

### Consulta de probabilidades

In [12]:
# P(A=0|C=0)
p = QABC.Query((A,), (C,), (0,), (0,))
p

0.3696682464454977

In [13]:
# P(A=0,B=0)
p = QABC.Query(vars=(A,B), vars_values=(0,0))
p

0.06

### Consultas de valores más probables

In [14]:
#P(A|B=0)
p = QABC.Query(vars=(A,), indep=(B,), indep_values=(0,))
p

(('A',), (1,), 0.7777777777777777)

In [15]:
#P(A=0|B)
p = QABC.Query(vars=(A,), indep=(B,), vars_values=(0,))
p

(('B',), (1,), 0.32876712328767127)

In [16]:
#P(AB)
p = QABC.Query(vars=(A,B))
p

(('A', 'B'), (1, 1), 0.48999999999999994)

# Problema de autoria de textos 

## Planteamiento

### Especificación

#### Variables

- $A$ : Variable para los autores
- $T$ : Variable para los tipos de textos
- $W_I$ : Variable para las palabras

##### Variable $A$

$|A| = 10 $

##### Variable $T$

$ T = \{'minicuento', 'fábula', 'cuento'\} $

##### Variable $W_i$

$ W_i = \{0,1\}$, donde cada $W_i$ representa una variable para cada palabra.

Si $W_i$ es 0, entonces la palabra no esta. Si $W_i$ es 1 entonces la palabra esta.

#### Modelo de descomposición

$P(A)P(T|A)\displaystyle \prod_{i=0}^{n} P(W_i|AT)$

## Implementación

### Obtención de datos para el test

In [ ]:
archivos = glob.glob('./Train/*/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_train = tp.lee_cuentos(archivos)

archivos = glob.glob('./Test/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_test = tp.lee_cuentos(archivos,test=True)

#### Data Frames de trabajo

In [ ]:
s = df_train.texto.str.cat(sep=' ').split()
print(len(s))
conteos=Counter(s)
nuevo_vocabulario = list(set([palabra for palabra in s if conteos[palabra] < 10 and conteos[palabra] > 5]))
print(len(nuevo_vocabulario))

In [ ]:
df_train_T = df_train.copy()
df_train_T['nuevo_texto'] = df_train_T.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in nuevo_vocabulario]).\
    apply(lambda x : ' '.join(x))
df_train_T['Conteos']=df_train_T.nuevo_texto.str.split().apply(Counter)


df_train_T.head()

In [ ]:
df_test_T = df_test.copy()
df_test_T['nuevo_texto']=df_test_T.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in nuevo_vocabulario]).\
    apply(lambda x : ' '.join(x))
df_test_T['nuevo_total']=df_test_T.nuevo_texto.str.split().apply(len)
df_test_T['Conteos']=df_test_T.nuevo_texto.str.split().apply(Counter)


df_test_T.head()